## Problem Statement

A US-based housing company named Surprise Housing has decided to enter the Australian market. The
company uses data analytics to purchase houses at a price below their actual values and flip them at a
higher price. For the same purpose, the company has collected a data set from the sale of houses in
Australia. 
From the given collected data set the company wants to understand the driving factors for house price in US

As part of the assignment we need to find out the<br>
1.Variables which are significant in predicting the price of a house<br>
2.How well those variables describe the price of a house<br>
3.Determine the optimal value of lambda for ridge and lasso regression

In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
#load the data
housing=pd.read_csv("train.csv")
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 1. Data Understanding

In [4]:
#get basic info about dataset
housing.shape

(1460, 81)

In [5]:
housing.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
housing.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
housing.info

<bound method DataFrame.info of         Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Ut

In [8]:
housing.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

##### As per the given problem statement of predicting house prices, we infer that SalePrice is the target variable

##### Checking distribution of target Variable

In [9]:
# Distribution of target variable
plot_1 = sns.distplot(housing['SalePrice'])
    
plot_1.set(xlabel='SalePrice', ylabel='Frequency')
    
plot_1.set_title('SalePrice vs Frequency',fontsize =18)

plt.show()

<Figure size 640x480 with 1 Axes>

##### From the above distribution plot we see that the SalePrice has a lot of outliers

##### Verifying the datatype of all numeric columns

In [10]:
#list all numeric columns
housing.select_dtypes(exclude = ['object'])

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [11]:
#check datatype of Fireplaces
housing['Fireplaces'].dtype

dtype('int64')

As per the data dictionary Fireplaces is a categorical variable but it has been marked as float64.So we should convert the datatype of Fireplaces to Object

In [12]:
#change the datatype of Fireplaces to category
housing['Fireplaces']=housing['Fireplaces'].astype('category')
housing['Fireplaces'].dtype

CategoricalDtype(categories=[0, 1, 2, 3], ordered=False)

In [13]:
#list all non-numeric columns
#list all numeric columns
housing.select_dtypes(exclude = ['int64','float64']).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')

In [14]:
housing.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [15]:
#checking datatype of other categorical variables
housing[['BsmtFullBath', 'BsmtHalfBath', 'FullBath','HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','TotRmsAbvGrd']].dtypes

BsmtFullBath    int64
BsmtHalfBath    int64
FullBath        int64
HalfBath        int64
BedroomAbvGr    int64
KitchenAbvGr    int64
TotRmsAbvGrd    int64
dtype: object

In [16]:
#Since the above categorical variables are set as int64 , we need to convert them to object
catvar=['BsmtFullBath', 'BsmtHalfBath', 'FullBath','HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','TotRmsAbvGrd']
housing[catvar]=housing[catvar].astype('category')
housing[catvar].dtypes

BsmtFullBath    category
BsmtHalfBath    category
FullBath        category
HalfBath        category
BedroomAbvGr    category
KitchenAbvGr    category
TotRmsAbvGrd    category
dtype: object

## 2. Data Cleaning and Modification

### Creating Derived Columns

##### Age of the house is a significant driving factor for house price.Age of a house can be derived as (Year Sold-Year Built)

In [17]:
housing['Age']=housing['YrSold']-housing['YearBuilt']
housing['Age'].head()

0     5
1    31
2     7
3    91
4     8
Name: Age, dtype: int64

We can now remove YrSold and YearBuilt columns.Also YearRemodAdd is not very significant in influencing the house price.Hence we can drop that as well.

In [18]:
housing =housing.drop(['YrSold','YearBuilt','YearRemodAdd'],axis=1)

### Check for missing values

#### Check for missing values in columns 

In [19]:
#Check for missing values
# percentage of the missing values (column-wise)
round(100*(housing.isnull().sum()/len(housing.index)), 2)

Id                0.00
MSSubClass        0.00
MSZoning          0.00
LotFrontage      17.74
LotArea           0.00
                 ...  
MoSold            0.00
SaleType          0.00
SaleCondition     0.00
SalePrice         0.00
Age               0.00
Length: 79, dtype: float64

In [20]:
#dropping columns with more than 80% missing values
housing=housing.dropna(thresh=housing.shape[0]*0.8,how='all',axis=1)

In [21]:
#checking the shape
housing.shape

(1460, 74)

##### So we see that now we are left with 75 columns

In [22]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,SaleType,SaleCondition,SalePrice,Age
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,WD,Normal,208500,5
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,WD,Normal,181500,31
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,WD,Normal,223500,7
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,WD,Abnorml,140000,91
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,WD,Normal,250000,8


In [23]:
#checking the columns
housing.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch'

##### SurpriseHousing wants to find the most important driving factors for determining the house price

##### From the given dataset we see that 'Id' is just an ordinal column and doesnt contribute in driving house price.Hence we can drop Id

##### The column MiscVal have too many unique values.Hence we can drop them.
##### The column Condition2 is similar and mostly explained by the variable Condition1.Hence we can drop Condition2.
##### The column 'ExterCond' and 'ExterQual' convey similar information.Hence we can drop one of them.Similarly dropping variable Exterior2nd,BsmtQual,BsmtFinType2,GarageQual
##### The columns 'GarageYrBlt','3SsnPorch','ScreenPorch' have less significance in terms of house price compared to other variables.Hence dropping them


In [24]:
#dropping the columns
housing=housing.drop(['Id','MiscVal','Condition2','ExterCond','GarageYrBlt','3SsnPorch','ScreenPorch','Exterior2nd','BsmtQual','BsmtFinType2','GarageQual'],axis = 1)

In [25]:
housing.shape

(1460, 63)

##### So now we are left with 69 columns

In [26]:
#Sorting columns with missing values in descending order of missing percentage
percent_missing = housing.isnull().sum() * 100 / len(housing)
missing_value_df = pd.DataFrame({'column_name': housing.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

,column_name,percent_missing
LotFrontage,LotFrontage,17.739726
GarageCond,GarageCond,5.547945
GarageType,GarageType,5.547945
GarageFinish,GarageFinish,5.547945
BsmtExposure,BsmtExposure,2.602740
...,...,...
BsmtUnfSF,BsmtUnfSF,0.000000
TotalBsmtSF,TotalBsmtSF,0.000000
Heating,Heating,0.000000
MSZoning,MSZoning,0.000000


##### Missing values in LotFrontage

In [27]:
housing.LotFrontage.dtype
housing.LotFrontage.value_counts()

60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
137.0      1
141.0      1
38.0       1
140.0      1
46.0       1
Name: LotFrontage, Length: 110, dtype: int64

In [28]:
housing.LotFrontage.isnull().sum()

259

In [29]:
#inspecting the LotFrontage column
housing.LotFrontage.describe()

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [30]:
housing['LotFrontage']=housing['LotFrontage'].fillna(housing['LotFrontage'].median())

In [31]:
#checking LotFrontage for null values
housing.LotFrontage.isnull().sum()

0

In [32]:
housing.LotFrontage.describe()

count    1460.000000
mean       69.863699
std        22.027677
min        21.000000
25%        60.000000
50%        69.000000
75%        79.000000
max       313.000000
Name: LotFrontage, dtype: float64

##### Missing values in GarageType

In [33]:
#inspecting GarageType
housing.GarageType.value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [34]:
housing.GarageType.describe()

count       1379
unique         6
top       Attchd
freq         870
Name: GarageType, dtype: object

In [35]:
housing.GarageType.isnull().sum()

81

In [36]:
#checking values of GarageType where GarageArea is zero(implying there is no garage)
housing.loc[housing['GarageArea']==0,'GarageType']

39      NaN
48      NaN
78      NaN
88      NaN
89      NaN
       ... 
1349    NaN
1407    NaN
1449    NaN
1450    NaN
1453    NaN
Name: GarageType, Length: 81, dtype: object

##### Here we see that the missing values in GarageType are for the houses where there is no garage.Hence we are imputing or replacing such missing values in GarageType with 'None'

In [37]:
#filling missing values in GarageType with 'None'
housing['GarageType']=housing['GarageType'].fillna(value='None')


In [38]:
#checking GarageType for null values
housing.GarageType.isnull().sum()

0

##### Missing Values in GarageCond

In [39]:
#inspecting the GarageCond column
housing.GarageCond.describe()

count     1379
unique       5
top         TA
freq      1326
Name: GarageCond, dtype: object

In [40]:
housing.GarageCond.value_counts()

TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: GarageCond, dtype: int64

In [41]:
housing.GarageCond.isnull().sum()

81

In [42]:
#checking values of GarageCond where GarageArea is zero(implying there is no garage)
housing.loc[housing['GarageArea']==0,'GarageCond']

39      NaN
48      NaN
78      NaN
88      NaN
89      NaN
       ... 
1349    NaN
1407    NaN
1449    NaN
1450    NaN
1453    NaN
Name: GarageCond, Length: 81, dtype: object

##### Here we see that the missing values in GarageCond are for the houses where there is no garage.Hence we are imputing or replacing such missing values in GarageCond with 'None'

In [43]:
#filling missing values in GarageCond with 'None'
housing['GarageCond']=housing['GarageCond'].fillna(value='None')

In [44]:
#checking GarageCond for null values
housing.GarageCond.isnull().sum()

0

##### Missing values in GarageFinish

In [45]:
# inspecting the GarageFinish column
housing['GarageFinish'].value_counts()

Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64

In [46]:
housing.GarageFinish.describe()

count     1379
unique       3
top        Unf
freq       605
Name: GarageFinish, dtype: object

In [47]:
housing.GarageFinish.isnull().sum()

81

In [48]:
#checking values of GarageFinish where GarageArea is zero(implying there is no garage)
housing.loc[housing['GarageArea']==0,'GarageFinish']

39      NaN
48      NaN
78      NaN
88      NaN
89      NaN
       ... 
1349    NaN
1407    NaN
1449    NaN
1450    NaN
1453    NaN
Name: GarageFinish, Length: 81, dtype: object

##### Here we see that the missing values in GarageFinish are for the houses where there is no garage.Hence we are imputing or replacing such missing values in GarageFinish with 'None'

In [49]:
#filling missing values in GarageFinish with 'None'
housing['GarageFinish']=housing['GarageFinish'].fillna(value='None')

In [50]:
#checking GarageFinish for null values
housing.GarageFinish.isnull().sum()

0

##### Checking for remaining columns with missing values

In [51]:
#Sorting columns with missing values in descending order of missing percentage
percent_missing = housing.isnull().sum() * 100 / len(housing)
missing_value_df = pd.DataFrame({'column_name': housing.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

,column_name,percent_missing
BsmtExposure,BsmtExposure,2.602740
BsmtFinType1,BsmtFinType1,2.534247
BsmtCond,BsmtCond,2.534247
MasVnrType,MasVnrType,0.547945
MasVnrArea,MasVnrArea,0.547945
...,...,...
BsmtUnfSF,BsmtUnfSF,0.000000
TotalBsmtSF,TotalBsmtSF,0.000000
Heating,Heating,0.000000
MSZoning,MSZoning,0.000000


##### Missing values in BsmtExposure

In [52]:
housing.BsmtExposure.value_counts()

No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64

In [53]:
housing.BsmtExposure.isnull().sum()

38

In [54]:
#filling missing values with highest occuring value(No)
housing['BsmtExposure']=housing['BsmtExposure'].fillna(value='No')

In [55]:
housing.TotalBsmtSF.value_counts()

0       37
864     35
672     17
912     15
1040    14
        ..
1838     1
1581     1
707      1
611      1
1542     1
Name: TotalBsmtSF, Length: 721, dtype: int64

##### From above we see that 37 houses have no basement

##### Missing values in BsmtFinType1

In [56]:
housing.BsmtFinType1.isnull().sum()

37

In [57]:
#checking values of BsmtFinType1 where TotalBsmtSF is zero(implying there is no basement)
len(housing.loc[housing['TotalBsmtSF']==0,'BsmtFinType1'])

37

In [58]:
#filling all  missing values of BsmtFinType1 with 'No'
housing['BsmtFinType1']=housing['BsmtFinType1'].fillna(value='No')

In [59]:
#Sorting columns with missing values in descending order of missing percentage
percent_missing = housing.isnull().sum() * 100 / len(housing)
missing_value_df = pd.DataFrame({'column_name': housing.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

,column_name,percent_missing
BsmtCond,BsmtCond,2.534247
MasVnrType,MasVnrType,0.547945
MasVnrArea,MasVnrArea,0.547945
Electrical,Electrical,0.068493
BedroomAbvGr,BedroomAbvGr,0.000000
...,...,...
BsmtFinSF2,BsmtFinSF2,0.000000
BsmtUnfSF,BsmtUnfSF,0.000000
TotalBsmtSF,TotalBsmtSF,0.000000
Heating,Heating,0.000000


##### Missing values in BsmtCond

In [60]:
housing.BsmtCond.isnull().sum()

37

In [61]:
#checking values of BsmtCond where TotalBsmtSF is zero(implying there is no basement)
len(housing.loc[housing['TotalBsmtSF']==0,'BsmtCond'])

37

In [62]:
#filling all missing values of BsmtCond with 'No'
housing['BsmtCond']=housing['BsmtCond'].fillna(value='No')

In [63]:
housing.MasVnrArea.value_counts()

0.0      861
180.0      8
72.0       8
108.0      8
120.0      7
        ... 
562.0      1
89.0       1
921.0      1
762.0      1
119.0      1
Name: MasVnrArea, Length: 327, dtype: int64

##### From above we see that 861 houses have no masonry veneer area

##### Missing values in MasVnrType

In [64]:
housing.MasVnrType.value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [65]:
housing.MasVnrType.isnull().sum()

8

In [66]:
#filling all missing values of MasVnrType with 'No'
housing['MasVnrType']=housing['MasVnrType'].fillna(value='No')

In [67]:
housing.MasVnrType.isnull().sum()

0

##### Missing values in MasVnrArea

In [68]:
housing.MasVnrArea.isnull().sum()

8

In [69]:
housing.MasVnrArea.describe()

count    1452.000000
mean      103.685262
std       181.066207
min         0.000000
25%         0.000000
50%         0.000000
75%       166.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

In [70]:
#filling all missing values of MasVnrArea with median
housing['MasVnrArea']=housing['MasVnrArea'].fillna(housing['MasVnrArea'].median())

In [71]:
#Sorting columns with missing values in descending order of missing percentage
percent_missing = housing.isnull().sum() * 100 / len(housing)
missing_value_df = pd.DataFrame({'column_name': housing.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)
missing_value_df

,column_name,percent_missing
Electrical,Electrical,0.068493
Fireplaces,Fireplaces,0.000000
1stFlrSF,1stFlrSF,0.000000
2ndFlrSF,2ndFlrSF,0.000000
LowQualFinSF,LowQualFinSF,0.000000
...,...,...
BsmtFinSF1,BsmtFinSF1,0.000000
BsmtFinSF2,BsmtFinSF2,0.000000
BsmtUnfSF,BsmtUnfSF,0.000000
TotalBsmtSF,TotalBsmtSF,0.000000


In [72]:
housing.shape

(1460, 63)

## 3. Visualising the data

### Univariate Analysis

### Univariate Analysis of Numerical data and checking for outliers in continuous variables

In [73]:
housing.SalePrice

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [74]:
housing.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'Foundation', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'PoolArea', 'MoSold',
       'SaleType', 'SaleCondition', 'SalePrice', 'Age'],
      dtype='object')

In [75]:
#  Numerical Data
housing_NUM = ['SalePrice','Age','LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF', 'EnclosedPorch', 'PoolArea']

#  plot Numerical Data
a = 18  # number of rows
b = 3  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(20,95))

for i in housing_NUM:
    plt.subplot(a, b, c)
    plt.title('{} (dist)'.format(i, a, b, c))
    plt.xlabel(i)
    sns.distplot(housing[i])
    c = c + 1

    plt.subplot(a, b, c)
    plt.title('{} (box)'.format(i, a, b, c))
    plt.xlabel(i)
    plt.boxplot(x = housing[i])
    c = c + 1

    plt.subplot(a, b, c)
    plt.title('{} (scatter)'.format(i, a, b, c))
    plt.xlabel(i)
    sns.scatterplot(data = housing, x = i, y = 'SalePrice')
    c = c + 1

plt.show()

<Figure size 2000x9500 with 54 Axes>

##### From Above we see that the variables PoolArea,EnclosedPorch,OpenPorchSF,WoodDeckSF,LowQualFinSF,2ndFlrSF,BsmtFinSF2,MasVnrArea are heavily skewed.Hence we are dropping those variables

In [76]:
#dropping the columns
housing=housing.drop(['PoolArea','EnclosedPorch','OpenPorchSF','WoodDeckSF','LowQualFinSF','2ndFlrSF','BsmtFinSF2','MasVnrArea'],axis = 1)

#### Checking for outliers

From the above box plots we see that the variables 'SalePrice','Age','LotFrontage','LotArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea' have a lot of outliers

#### Removing Outliers using IQR

In [77]:
def subset_by_iqr(df, column, whisker_width=1.5):
   # Calculate Q1, Q2 and IQR
    q1 = df[column].quantile(0.25)                 
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
    return df.loc[filter]                                                     



housing = subset_by_iqr(housing, 'SalePrice', whisker_width=1.5)
housing = subset_by_iqr(housing, 'Age', whisker_width=1.5)
housing = subset_by_iqr(housing, 'LotFrontage', whisker_width=1.5)
housing = subset_by_iqr(housing, 'LotArea', whisker_width=1.5)
housing = subset_by_iqr(housing, 'BsmtFinSF1', whisker_width=1.5)
housing = subset_by_iqr(housing, 'TotalBsmtSF', whisker_width=1.5)
housing = subset_by_iqr(housing, '1stFlrSF', whisker_width=1.5)
housing = subset_by_iqr(housing, 'GrLivArea', whisker_width=1.5)
housing.shape

(1155, 55)

In [78]:
housing.SalePrice

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1454    185000
1455    175000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1155, dtype: int64

##### Applying Data Transformation

In [79]:
#logcolumns = ['SalePrice', 'LotFrontage', 'LotArea','1stFlrSF', 'TotalBsmtSF', 'GrLivArea']
#housing.loc[:, logcolumns] = np.log(housing[logcolumns])
housing["TransformedPrice"] = np.log(housing["SalePrice"])

### Univariate Analysis of Categorical data

In [80]:
housing.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType',
       'ExterQual', 'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageCond', 'PavedDrive',
       'MoSold', 'SaleType', 'SaleCondition', 'SalePrice', 'Age',
       'TransformedPrice'],
      dtype='object')

In [81]:
housing_catvar=['MSSubClass','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope', 'Neighborhood','Condition1','BldgType','HouseStyle','OverallQual','OverallCond','RoofStyle','RoofMatl','Exterior1st','MasVnrType','ExterQual','Foundation','BsmtCond','BsmtExposure','BsmtFinType1','Heating','HeatingQC','CentralAir','Electrical','BsmtFullBath','BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','GarageType','GarageFinish','GarageCars','GarageCond','PavedDrive','MoSold','SaleType','SaleCondition']
#checking count of categorical variables
print(len(housing_catvar))

44


In [82]:
#  Categorical Data
#  Categorical Data
a = 44  # number of rows
b = 4  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(20,230))

for i in housing_catvar:
    plt.subplot(a, b, c)
    plt.title('{}'.format(i))
    plt.xlabel(i)
    sns.barplot(x = i, y = 'SalePrice',data = housing )
    c = c + 1

plt.show()


<Figure size 2000x23000 with 44 Axes>

In [83]:
housing.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType',
       'ExterQual', 'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageCond', 'PavedDrive',
       'MoSold', 'SaleType', 'SaleCondition', 'SalePrice', 'Age',
       'TransformedPrice'],
      dtype='object')

In [84]:
# Let's see the correlation matrix 
plt.figure(figsize = (40,37))        # Size of the figure
#sns.heatmap(housing.corr(),annot = True)
#plt.show()


<Figure size 4000x3700 with 0 Axes>

SalePrice is highly correlated with Age of the house.<br>
highly correlated predictor variables are OverallQual,'BsmtFinSF1', 'TotalBsmtSF','GarageCars', 'GarageArea'.So dropping them


In [85]:
housing=housing.drop(['OverallQual','BsmtFinSF1', 'TotalBsmtSF','GarageCars', 'GarageArea'],axis=1)

## 4. Data Preparation

### Creating Dummy Variables

##### Linear Regression does not accept categorical variables.So we need to create dummy numeric variables

#### Grouping the categorical variables

In [86]:
housing.SalePrice

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1454    185000
1455    175000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1155, dtype: int64

In [87]:
housing.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'OverallCond',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'ExterQual',
       'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtUnfSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageFinish', 'GarageCond',
       'PavedDrive', 'MoSold', 'SaleType', 'SaleCondition', 'SalePrice', 'Age',
       'TransformedPrice'],
      dtype='object')

##### Among the categorical variables , we see that the variables Neighborhood,Condition1,HouseStyle,OverallQual,OverallCond,RoofMatl,Exterior1st,BsmtFinType1,BedroomAbvGr,TotRmsAbvGrd,Functional,GarageType,SaleType,SaleCondition have lot of categories with very less values. So we are grouping the categories with values less than 5% into 'Other'

In [88]:
#grouping categories into 'Other' and relabelling columns for MSSubClass,OverallQual,OverallCond
housing['MSSubClass']=housing['MSSubClass'].mask(housing['MSSubClass'].map(housing['MSSubClass'].value_counts(normalize=True)) < 0.05, 'Other')
#housing['OverallQual']=housing['OverallQual'].mask(housing['OverallQual'].map(housing['OverallQual'].value_counts(normalize=True)) < 0.05, 'Other')
housing['OverallCond']=housing['OverallCond'].mask(housing['OverallCond'].map(housing['OverallCond'].value_counts(normalize=True)) < 0.05, 'Other')

#checking the available values to relabel for MSSubClass
housing.MSSubClass.value_counts(normalize=True)
cleanup_MSSubClass={"MSSubClass":{20: "1-STORY-NEW",30: "1-STORY-OLD",60:"2-STORY-NEW",50:"1-1/2-STORY-FINISHED",120:"1-STORY-PUD"}}
housing = housing.replace(cleanup_MSSubClass)

#checking the available values to relabel for OverallQual
#housing.OverallQual.value_counts(normalize=True)
##cleanup_OverallQual={"OverallQual":{5: "Average",6: "Above_Average",7:"Good",8:"Very Good",4:"Below Average"}}
#housing = housing.replace(cleanup_OverallQual)
#housing.OverallQual.value_counts(normalize=True)

#checking the available values to relabel for OverallCond
housing.OverallCond.value_counts(normalize=True)
cleanup_OverallCond={"OverallCond":{5: "Average",6: "Above_Average",7:"Good",8:"Very Good"}}
housing = housing.replace(cleanup_OverallCond)
housing.OverallCond.value_counts(normalize=True)

Average          0.545455
Above_Average    0.188745
Good             0.143723
Other            0.065801
Very Good        0.056277
Name: OverallCond, dtype: float64

In [89]:
#grouping the rare values of the remaining categorical variable to Other
housing['MSZoning']=housing['MSZoning'].mask(housing['MSZoning'].map(housing['MSZoning'].value_counts(normalize=True)) < 0.05, 'Other')
housing['Neighborhood']=housing['Neighborhood'].mask(housing['Neighborhood'].map(housing['Neighborhood'].value_counts(normalize=True)) < 0.05, 'Other')
housing['Condition1']=housing['Condition1'].mask(housing['Condition1'].map(housing['Condition1'].value_counts(normalize=True)) < 0.05, 'Other')
housing['HouseStyle']=housing['HouseStyle'].mask(housing['HouseStyle'].map(housing['HouseStyle'].value_counts(normalize=True)) < 0.05, 'Other')
housing['RoofMatl']=housing['RoofMatl'].mask(housing['RoofMatl'].map(housing['RoofMatl'].value_counts(normalize=True)) < 0.05, 'Other')
housing['Exterior1st']=housing['Exterior1st'].mask(housing['Exterior1st'].map(housing['Exterior1st'].value_counts(normalize=True)) < 0.05, 'Other')
housing['BsmtFinType1']=housing['BsmtFinType1'].mask(housing['BsmtFinType1'].map(housing['BsmtFinType1'].value_counts(normalize=True)) < 0.05, 'Other')
housing['BedroomAbvGr']=housing['SaleCondition'].mask(housing['SaleCondition'].map(housing['SaleCondition'].value_counts(normalize=True)) < 0.05, 'Other')
housing['TotRmsAbvGrd']=housing['SaleCondition'].mask(housing['SaleCondition'].map(housing['SaleCondition'].value_counts(normalize=True)) < 0.05, 'Other')
housing['Functional']=housing['Functional'].mask(housing['Functional'].map(housing['Functional'].value_counts(normalize=True)) < 0.05, 'Other')
housing['GarageType']=housing['GarageType'].mask(housing['GarageType'].map(housing['GarageType'].value_counts(normalize=True)) < 0.05, 'Other')
housing['SaleType']=housing['SaleType'].mask(housing['SaleType'].map(housing['SaleType'].value_counts(normalize=True)) < 0.05, 'Other')
housing['SaleCondition']=housing['SaleCondition'].mask(housing['SaleCondition'].map(housing['SaleCondition'].value_counts(normalize=True)) < 0.05, 'Other')


#### For categorical variables with multiple levels, create dummy features (one-hot encoded)

In [90]:
print(housing.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'OverallCond',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'ExterQual',
       'Foundation', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtUnfSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageFinish', 'GarageCond',
       'PavedDrive', 'MoSold', 'SaleType', 'SaleCondition', 'SalePrice', 'Age',
       'TransformedPrice'],
      dtype='object')


In [91]:
# Creating a dummy variable for  the categorical variables and dropping the first one.
housing_catvar1=['MSSubClass','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope', 'Neighborhood','Condition1','BldgType','HouseStyle','OverallCond','RoofStyle','RoofMatl','Exterior1st','MasVnrType','ExterQual','Foundation','BsmtCond','BsmtExposure','BsmtFinType1','Heating','HeatingQC','CentralAir','Electrical','BsmtFullBath','BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','GarageType','GarageFinish','GarageCond','PavedDrive','MoSold','SaleType','SaleCondition']

dummy1 = pd.get_dummies(housing[housing_catvar1],drop_first=True)
# Adding the results to the master dataframe
housing = pd.concat([housing, dummy1], axis=1)
housing.columns


Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       ...
       'GarageCond_None', 'GarageCond_Po', 'GarageCond_TA', 'PavedDrive_P',
       'PavedDrive_Y', 'SaleType_Other', 'SaleType_WD', 'SaleCondition_Normal',
       'SaleCondition_Other', 'SaleCondition_Partial'],
      dtype='object', length=180)

##### Dropping the repeated variables

In [92]:
housing=housing.drop(housing_catvar1,axis=1)

In [93]:
housing.SalePrice

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1454    185000
1455    175000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1155, dtype: int64

Now we can see all variables are numeric

In [94]:
housing.shape


(1155, 137)

##### end of eda

### Test-Train Split

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
#Putting feature variables to X
X=housing.drop([ "SalePrice", "TransformedPrice"], axis=1)

X.head()

,LotFrontage,LotArea,BsmtUnfSF,1stFlrSF,GrLivArea,Fireplaces,Age,MSSubClass_1-STORY-NEW,MSSubClass_1-STORY-OLD,MSSubClass_1-STORY-PUD,...,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_Other,SaleType_WD,SaleCondition_Normal,SaleCondition_Other,SaleCondition_Partial
0,65.0,8450,150,856,1710,0,5,0,0,0,...,0,0,1,0,1,0,1,1,0,0
1,80.0,9600,284,1262,1262,1,31,1,0,0,...,0,0,1,0,1,0,1,1,0,0
2,68.0,11250,434,920,1786,1,7,0,0,0,...,0,0,1,0,1,0,1,1,0,0
3,60.0,9550,540,961,1717,1,91,0,0,0,...,0,0,1,0,1,0,1,0,0,0
4,84.0,14260,490,1145,2198,1,8,0,0,0,...,0,0,1,0,1,0,1,1,0,0


In [97]:
#Putting response variable to y
y=housing['TransformedPrice']
y.head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: TransformedPrice, dtype: float64

In [98]:
# split into train and test
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

### Feature Scaling

In [99]:
from sklearn.preprocessing import StandardScaler

In [100]:
#housing_NUM = ['SalePrice','Age','LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF', 'EnclosedPorch', 'PoolArea']
print(X_train.columns)

Index(['LotFrontage', 'LotArea', 'BsmtUnfSF', '1stFlrSF', 'GrLivArea',
       'Fireplaces', 'Age', 'MSSubClass_1-STORY-NEW', 'MSSubClass_1-STORY-OLD',
       'MSSubClass_1-STORY-PUD',
       ...
       'GarageCond_None', 'GarageCond_Po', 'GarageCond_TA', 'PavedDrive_P',
       'PavedDrive_Y', 'SaleType_Other', 'SaleType_WD', 'SaleCondition_Normal',
       'SaleCondition_Other', 'SaleCondition_Partial'],
      dtype='object', length=135)


In [101]:
scaler=StandardScaler()
varstoscale=['Age','LotFrontage', 'LotArea','BsmtUnfSF','1stFlrSF','GrLivArea']
X_train[varstoscale]=scaler.fit_transform(X_train[varstoscale])
X_train.head()

,LotFrontage,LotArea,BsmtUnfSF,1stFlrSF,GrLivArea,Fireplaces,Age,MSSubClass_1-STORY-NEW,MSSubClass_1-STORY-OLD,MSSubClass_1-STORY-PUD,...,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_Other,SaleType_WD,SaleCondition_Normal,SaleCondition_Other,SaleCondition_Partial
550,-1.071694,-1.918220,-0.100460,-0.111652,-0.808975,0,-0.221663,0,0,1,...,0,0,1,0,1,1,0,0,0,0
578,-2.466577,-2.086924,0.339568,-1.369846,-0.037348,0,-1.222989,0,0,0,...,0,0,1,0,1,0,1,0,0,0
395,0.029530,0.206145,-0.358577,0.146174,-0.629805,0,0.546020,1,0,0,...,0,0,1,0,1,0,1,1,0,0
505,-0.557790,-0.552831,1.005755,-0.486360,1.319569,0,0.646152,0,0,0,...,0,0,1,0,0,1,0,1,0,0
1306,-1.438768,-0.799162,2.008723,0.916216,-0.094682,1,-1.222989,0,0,1,...,0,0,1,0,1,0,0,0,0,1


In [102]:
# Let's see the correlation matrix 
#plt.figure(figsize = (40,37))        # Size of the figure
#sns.heatmap(housing.corr(),annot = True)
#plt.show()

## 5. Model Building

### Feature Selection Using RFE


In [103]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
linreg = LinearRegression()

In [104]:
rfe = RFE(linreg, 30)             # running RFE with 13 variables as output
rfe = rfe.fit(X_train, y_train)

In [105]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

[('LotFrontage', False, 59),
 ('LotArea', False, 38),
 ('BsmtUnfSF', False, 95),
 ('1stFlrSF', False, 8),
 ('GrLivArea', True, 1),
 ('Fireplaces', False, 40),
 ('Age', True, 1),
 ('MSSubClass_1-STORY-NEW', False, 4),
 ('MSSubClass_1-STORY-OLD', False, 6),
 ('MSSubClass_1-STORY-PUD', True, 1),
 ('MSSubClass_2-STORY-NEW', False, 88),
 ('MSSubClass_Other', False, 52),
 ('MSZoning_RL', True, 1),
 ('MSZoning_RM', True, 1),
 ('Street_Pave', True, 1),
 ('LotShape_IR2', False, 85),
 ('LotShape_IR3', False, 37),
 ('LotShape_Reg', False, 79),
 ('LandContour_HLS', True, 1),
 ('LandContour_Low', False, 97),
 ('LandContour_Lvl', False, 80),
 ('Utilities_NoSeWa', True, 1),
 ('LotConfig_CulDSac', False, 55),
 ('LotConfig_FR2', False, 100),
 ('LotConfig_FR3', False, 60),
 ('LotConfig_Inside', False, 84),
 ('LandSlope_Mod', False, 71),
 ('LandSlope_Sev', True, 1),
 ('Neighborhood_Edwards', False, 14),
 ('Neighborhood_Gilbert', False, 56),
 ('Neighborhood_NAmes', False, 104),
 ('Neighborhood_NWAmes', Fa

In [106]:
col = X_train.columns[rfe.support_]
col

Index(['GrLivArea', 'Age', 'MSSubClass_1-STORY-PUD', 'MSZoning_RL',
       'MSZoning_RM', 'Street_Pave', 'LandContour_HLS', 'Utilities_NoSeWa',
       'LandSlope_Sev', 'Neighborhood_Somerst', 'BldgType_2fmCon',
       'BldgType_Twnhs', 'BldgType_TwnhsE', 'OverallCond_Other',
       'ExterQual_Fa', 'ExterQual_Gd', 'ExterQual_TA', 'BsmtCond_Po',
       'CentralAir_Y', 'Electrical_Mix', 'BsmtFullBath_2', 'BsmtHalfBath_2',
       'FullBath_1', 'FullBath_2', 'FullBath_3', 'KitchenAbvGr_1',
       'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA',
       'GarageCond_None'],
      dtype='object')

In [107]:
X_train_rfe=X_train[col]

In [108]:
#varstoscale=['Age','LotFrontage', 'LotArea','BsmtUnfSF','1stFlrSF','GrLivArea']
X_test[varstoscale]=scaler.transform(X_test[varstoscale])
X_test.head()

,LotFrontage,LotArea,BsmtUnfSF,1stFlrSF,GrLivArea,Fireplaces,Age,MSSubClass_1-STORY-NEW,MSSubClass_1-STORY-OLD,MSSubClass_1-STORY-PUD,...,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_Other,SaleType_WD,SaleCondition_Normal,SaleCondition_Other,SaleCondition_Partial
1239,-0.264130,0.000934,1.222081,1.314988,0.182434,1,-1.222989,1,0,0,...,0,0,1,0,1,0,1,1,0,0
442,-1.145109,-1.073931,1.295829,0.091171,0.395050,1,1.347080,0,0,0,...,0,0,1,1,0,0,1,1,0,0
538,0.102944,0.967812,-0.424950,0.197739,-0.593970,1,0.011980,1,0,0,...,0,0,1,0,1,0,1,1,0,0
355,2.745881,0.850987,-0.562612,0.338684,-0.496024,0,-0.855836,1,0,0,...,0,0,1,0,1,0,1,1,0,0
343,-0.337545,-0.071313,2.716701,2.002525,0.660222,1,-1.156234,0,0,1,...,0,0,1,0,1,0,1,1,0,0


In [109]:
col=X_test.columns[rfe.support_]
X_test_rfe=X_test[col]
X_test_rfe.columns
#print(col)

Index(['GrLivArea', 'Age', 'MSSubClass_1-STORY-PUD', 'MSZoning_RL',
       'MSZoning_RM', 'Street_Pave', 'LandContour_HLS', 'Utilities_NoSeWa',
       'LandSlope_Sev', 'Neighborhood_Somerst', 'BldgType_2fmCon',
       'BldgType_Twnhs', 'BldgType_TwnhsE', 'OverallCond_Other',
       'ExterQual_Fa', 'ExterQual_Gd', 'ExterQual_TA', 'BsmtCond_Po',
       'CentralAir_Y', 'Electrical_Mix', 'BsmtFullBath_2', 'BsmtHalfBath_2',
       'FullBath_1', 'FullBath_2', 'FullBath_3', 'KitchenAbvGr_1',
       'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA',
       'GarageCond_None'],
      dtype='object')

## Ridge and Lasso Regression

### Ridge Regression

In [110]:
#Importing all libraries
from sklearn import linear_model, metrics
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

We are using X_train_rfe as the training dataset

In [111]:
# list of alphas to tune - if value too high it will lead to underfitting, if it is too low, 
# it will not handle the overfitting
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'r2',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train_rfe, y_train) 

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='r2', verbose=1)

In [112]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head()
cv_results.loc[ :,['params','mean_test_score','mean_train_score']]

,params,mean_test_score,mean_train_score
0,{'alpha': 0.0001},0.793737,0.843414
1,{'alpha': 0.001},0.793833,0.843414
2,{'alpha': 0.01},0.794751,0.843406
3,{'alpha': 0.05},0.798180,0.843286
4,{'alpha': 0.1},0.801407,0.843070
5,{'alpha': 0.2},0.805805,0.842628
6,{'alpha': 0.3},0.808670,0.842223
7,{'alpha': 0.4},0.810676,0.841848
8,{'alpha': 0.5},0.812146,0.841494
9,{'alpha': 0.6},0.813258,0.841155


In [113]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')

plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

<Figure size 4000x3700 with 1 Axes>

In [114]:
# Printing the best hyperparameter alpha
print(model_cv.best_params_)

{'alpha': 2.0}


In [115]:
#Fitting Ridge model for alpha = 2.0 and printing coefficients which have been penalised
alpha = 1.0
ridge = Ridge(alpha=alpha)

ridge.fit(X_train_rfe, y_train)
print(ridge.coef_)

[ 0.15768784 -0.10054316  0.12645111  0.18455249  0.12398391  0.1444172
  0.06904582 -0.05769218 -0.15339943  0.19683057  0.18105095 -0.11569487
 -0.12031057 -0.14386299 -0.23410948 -0.13441175 -0.1936792  -0.02203541
  0.14971538 -0.15544586  0.18042386  0.06350256  0.09248043  0.09958375
  0.06928677  0.08601403 -0.20952177 -0.12606932 -0.20163393 -0.08960594]


In [116]:
# Lets calculate some metrics such as R2 score, RSS and RMSE
y_pred_train = ridge.predict(X_train_rfe)
y_pred_test = ridge.predict(X_test_rfe)

metric2 = []
r2_train_lr = r2_score(y_train, y_pred_train)
print(r2_train_lr)
metric2.append(r2_train_lr)

r2_test_lr = r2_score(y_test, y_pred_test)
print(r2_test_lr)
metric2.append(r2_test_lr)

rss1_lr = np.sum(np.square(y_train - y_pred_train))
print(rss1_lr)
metric2.append(rss1_lr)

rss2_lr = np.sum(np.square(y_test - y_pred_test))
print(rss2_lr)
metric2.append(rss2_lr)

mse_train_lr = mean_squared_error(y_train, y_pred_train)
print(mse_train_lr)
metric2.append(mse_train_lr**0.5)

mse_test_lr = mean_squared_error(y_test, y_pred_test)
print(mse_test_lr)
metric2.append(mse_test_lr**0.5)

0.8381232349693345
0.7699578454735796
14.497206185346418
9.420462629983685
0.017942086863052497
0.027148307291019268


### Lasso

In [117]:
lasso = Lasso()

# cross validation
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'r2', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train_rfe, y_train) 

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='r2', verbose=1)

In [118]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results.head()
cv_results.loc[ :,['params','mean_test_score','mean_train_score']]

,params,mean_test_score,mean_train_score
0,{'alpha': 0.0001},0.806682,0.842155
1,{'alpha': 0.001},0.804664,0.820829
2,{'alpha': 0.01},0.747503,0.753162
3,{'alpha': 0.05},0.695795,0.696887
4,{'alpha': 0.1},0.588159,0.588121
5,{'alpha': 0.2},0.150905,0.153055
6,{'alpha': 0.3},-0.004077,0.000000
7,{'alpha': 0.4},-0.004077,0.000000
8,{'alpha': 0.5},-0.004077,0.000000
9,{'alpha': 0.6},-0.004077,0.000000


##### alpha=0.001 seems to be a sweet spot

In [119]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('r2')

plt.title("R2 and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

In [120]:
# Printing the best hyperparameter alpha
print(model_cv.best_params_)

{'alpha': 0.0001}


In [121]:
#Fitting Lasso model for alpha = .001 and printing coefficients which have been penalised

alpha =.001

lasso = Lasso(alpha=alpha)
        
lasso.fit(X_train_rfe, y_train) 

Lasso(alpha=0.001)

In [122]:
lasso.coef_

array([ 0.15950528, -0.10423018,  0.02119159,  0.12528959,  0.0538473 ,
        0.        ,  0.02480132, -0.        , -0.        ,  0.11618424,
        0.08781812, -0.        , -0.01847706, -0.14362894, -0.08343357,
       -0.        , -0.06416625, -0.        ,  0.15458058, -0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.00880478, -0.17402164, -0.09137528, -0.1692033 , -0.08292119])

In [123]:
# Lets calculate some metrics such as R2 score, RSS and RMSE

y_pred_train = lasso.predict(X_train_rfe)
y_pred_test = lasso.predict(X_test_rfe)

metric3 = []
r2_train_lr = r2_score(y_train, y_pred_train)
print(r2_train_lr)
metric3.append(r2_train_lr)

r2_test_lr = r2_score(y_test, y_pred_test)
print(r2_test_lr)
metric3.append(r2_test_lr)

rss1_lr = np.sum(np.square(y_train - y_pred_train))
print(rss1_lr)
metric3.append(rss1_lr)

rss2_lr = np.sum(np.square(y_test - y_pred_test))
print(rss2_lr)
metric3.append(rss2_lr)

mse_train_lr = mean_squared_error(y_train, y_pred_train)
print(mse_train_lr)
metric3.append(mse_train_lr**0.5)

mse_test_lr = mean_squared_error(y_test, y_pred_test)
print(mse_test_lr)
metric3.append(mse_test_lr**0.5)

0.8188775071744037
0.7718625004855417
16.22079686852637
9.342465049930851
0.0200752436491663
0.026923530403258938


In [124]:
# Creating a table which contain all the metrics

metric_table = {'Metric': ['R2 Score (Train)','R2 Score (Test)','RSS (Train)','RSS (Test)',
                       'MSE (Train)','MSE (Test)']       
        }

metrictbl = pd.DataFrame(metric_table ,columns = ['Metric'] )

rg_metric = pd.Series(metric2, name = 'Ridge Regression')
ls_metric = pd.Series(metric3, name = 'Lasso Regression')

final_metric = pd.concat([metrictbl,rg_metric, ls_metric], axis = 1)

final_metric

,Metric,Ridge Regression,Lasso Regression
0,R2 Score (Train),0.838123,0.818878
1,R2 Score (Test),0.769958,0.771863
2,RSS (Train),14.497206,16.220797
3,RSS (Test),9.420463,9.342465
4,MSE (Train),0.133948,0.141687
5,MSE (Test),0.164767,0.164084


## Lets observe the changes in the coefficients after regularization

In [125]:
betas = pd.DataFrame(index= X_train.columns[rfe.support_])

In [126]:
betas.rows = X_train.columns[rfe.support_]

In [127]:

betas['Ridge'] = ridge.coef_
betas['Lasso'] = lasso.coef_

In [128]:
pd.set_option('display.max_rows', None)
betas.head(30)

,Ridge,Lasso
GrLivArea,0.157688,0.159505
Age,-0.100543,-0.104230
MSSubClass_1-STORY-PUD,0.126451,0.021192
MSZoning_RL,0.184552,0.125290
MSZoning_RM,0.123984,0.053847
Street_Pave,0.144417,0.000000
LandContour_HLS,0.069046,0.024801
Utilities_NoSeWa,-0.057692,-0.000000
LandSlope_Sev,-0.153399,-0.000000
Neighborhood_Somerst,0.196831,0.116184


## Interpretation

#####  Ridge Regression: Variables significant in predicting the price of a house

 The top most significant variables in Ridge are KitchenQual_TA,KitchenQual_Fa,CentralAir_Y,GrLivArea,Neighborhood_Somerst,

From the above Lasso regression we see that the following variables are significant in predicting the house price<br>
1.GrLivArea: This variable is highly significant and positively related to the house price<br>
2.Age:This variable is highly significant and negatively related to the house price<br>
3.MSZoning_RL:This variable is highly significant and positively related to the house price<br>
4.MSZoning_RM<br>
5.LandContour_HLS<br>
6.Neighborhood_Somerst<br>
7.CentralAir_Y<br>
8.OverallCond_Other<br>
9.KitchenQual_Fa<br>
10.KitchenQual_TA<br>
11.GarageCond_None<br>

##### Optimal value of lambda 
##### RIDGE REGRESSION: 5.0
##### LASSO REGRESSION: 0.001